In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import datetime
import seaborn as sns
from pandas.tseries.offsets import MonthEnd

In [3]:
#Custom functions
def daterange(x,y):
    dates = pd.date_range(x,y, freq='d')
    return dates 

In [4]:
directory = os.getcwd()
print(directory)
folder = r"C:\Users\Shraddha.Mishra\OneDrive - Shell\Career_development\Projects\Ultimate Potential South Africa\OneDrive_1_7-25-2024\Sales Data"
# folder = 'Sales Data'

c:\Users\Shraddha.Mishra\OneDrive - Shell\Career_development\Projects\Ultimate Potential South Africa


In [5]:
path = os.path.join(directory,folder)
path

'C:\\Users\\Shraddha.Mishra\\OneDrive - Shell\\Career_development\\Projects\\Ultimate Potential South Africa\\OneDrive_1_7-25-2024\\Sales Data'

## Reading Sales Data

In [6]:
full_data = pd.DataFrame()
for root,dirs,files in os.walk(path):
    for file in files:
       if file.endswith(".csv"):
           print(os.path.join(root,file))
           df = pd.read_csv(os.path.join(root,file))
           full_data = pd.concat([df,full_data])           

C:\Users\Shraddha.Mishra\OneDrive - Shell\Career_development\Projects\Ultimate Potential South Africa\OneDrive_1_7-25-2024\Sales Data\Year_2019.csv
C:\Users\Shraddha.Mishra\OneDrive - Shell\Career_development\Projects\Ultimate Potential South Africa\OneDrive_1_7-25-2024\Sales Data\Year_2020.csv
C:\Users\Shraddha.Mishra\OneDrive - Shell\Career_development\Projects\Ultimate Potential South Africa\OneDrive_1_7-25-2024\Sales Data\Year_2021.csv
C:\Users\Shraddha.Mishra\OneDrive - Shell\Career_development\Projects\Ultimate Potential South Africa\OneDrive_1_7-25-2024\Sales Data\Year_2022.csv
C:\Users\Shraddha.Mishra\OneDrive - Shell\Career_development\Projects\Ultimate Potential South Africa\OneDrive_1_7-25-2024\Sales Data\Year_2023.csv
C:\Users\Shraddha.Mishra\OneDrive - Shell\Career_development\Projects\Ultimate Potential South Africa\OneDrive_1_7-25-2024\Sales Data\Year_2024.csv


In [7]:
full_data.shape

(2781090, 13)

In [8]:
full_data.dtypes

Company Code             object
Distribution Channel     object
BL Date                   int64
Sales Order Number        int64
Material Number           int64
Material Text            object
Movement Type             int64
Receiving Plant Code     object
Receiving Plant Name     object
Billing Document        float64
Ship To                   int64
Ship To Name             object
Sales Volumes in L15    float64
dtype: object

In [9]:
full_data['date'] = pd.to_datetime(full_data['BL Date'], format='%Y%m%d')

## Adding External Features

In [10]:
material_list = [400003103,400003118,400003139,400003151,400003154,400003159,400006090]

In [11]:
full_data = full_data[full_data['Material Number'].isin(material_list) ]

In [12]:
full_data.shape

(2624953, 14)

In [13]:
#Adding Holidays
folder = 'External Variables'
file = 'SA Holiday Calendar.xlsx'
holidays = pd.read_excel(os.path.join(directory,folder, file))

#Writing Holidays file
subfolder = 'EDA'
subfolder_1 = 'External Vars Processed'
filename = 'holidays.csv'

write_path = os.path.join(directory,subfolder,subfolder_1,filename)
holidays.to_csv(write_path)

# full_data means sales data of year
full_data_timeseries = pd.merge(full_data,holidays, left_on = 'date', right_on = 'Holiday', how = 'left')

full_data_timeseries = full_data_timeseries.drop('Holiday', axis = 1)
full_data_timeseries['Holiday Desc'] = full_data_timeseries['Holiday Desc'].fillna('NA')
full_data_timeseries['Holiday_Grouping'] = full_data_timeseries['Holiday_Grouping'].fillna('NA')
full_data_timeseries['Holiday Flag'] = full_data_timeseries['Holiday Flag'].fillna(0)

In [14]:
full_data_timeseries.shape

# full_data_timeseries.head()

(2624953, 17)

In [15]:
#Reading Lockdowns
folder = 'External Variables'
file = 'SA Lockdown.xlsx'
lockdowns = pd.read_excel(os.path.join(directory,folder, file))
lockdowns = lockdowns[~lockdowns['End Date'].isna()] 
lockdowns['Date_Range'] = lockdowns.apply(lambda x: daterange(x['Start Date'], x['End Date']), axis=1)

# Expand Lists into Rows: The explode() function is used to transform each element of a list-like column into its own row,
# duplicating the values of other columns accordingly.
lockdowns = lockdowns.explode('Date_Range')
lockdowns = lockdowns[['Province','Date_Range']]
lockdowns.drop_duplicates(inplace = True)

#Reading Provice
folder = 'Mapping Files'
file = 'SA_Site_Mapping.xlsx'
sites = pd.read_excel(os.path.join(directory,folder, file))
sites = sites[sites['Site Status'] != 'Inactive']
sites = sites[['Ship To Party Code','Plant Code','Province']]
sites.drop_duplicates(inplace = True)

#Writing Lockdowns
filename = 'lockdowns.csv'
write_path = os.path.join(directory,subfolder,subfolder_1,filename)
lockdowns.to_csv(write_path)

#writing Province
filename = 'province.csv'
write_path = os.path.join(directory,subfolder,subfolder_1,filename)
sites.to_csv(write_path)

#Adding Lockdowns
full_data_timeseries = pd.merge(full_data_timeseries,sites, left_on = ['Ship To','Receiving Plant Code'], right_on = ['Ship To Party Code','Plant Code'], how = 'left')
full_data_timeseries = pd.merge(full_data_timeseries,lockdowns, left_on = ['Province','date'], right_on = ['Province','Date_Range'], how = 'left')
full_data_timeseries = full_data_timeseries.drop('Ship To Party Code', axis = 1)
full_data_timeseries = full_data_timeseries.drop('Plant Code', axis = 1)
full_data_timeseries['LockDown Flag'] = full_data_timeseries['Date_Range'].apply(lambda x: 0 if pd.isnull(x) else 1)
full_data_timeseries = full_data_timeseries.drop('Date_Range', axis = 1)

In [16]:
full_data_timeseries.shape
# full_data_timeseries.head()

(2624953, 19)

In [17]:
#Reading Depot Mapping
folder = 'Mapping Files'
file = 'SA_Depot_Mapping.xlsx'
depots = pd.read_excel(os.path.join(directory,folder, file))[['Plant','Region']]
depots.drop_duplicates(inplace = True)

folder = 'Mapping Files'
file = 'SA_Material_Mapping.xlsx'
materials = pd.read_excel(os.path.join(directory,folder, file))[['Material Number','Grade Type']]
materials.drop_duplicates(inplace = True)

#Adding Region and Grade Type
full_data_timeseries = pd.merge(full_data_timeseries,depots, left_on = ['Receiving Plant Code'], right_on = ['Plant'], how = 'left')
full_data_timeseries = full_data_timeseries.drop('Plant', axis = 1)
full_data_timeseries = pd.merge(full_data_timeseries,materials, on = ['Material Number'], how = 'left')

In [18]:
full_data_timeseries.shape
# done till here 

(2624953, 21)

In [19]:
#Reading Price Details
folder = 'External Variables'
file = 'SA Price Information.xlsx'
price = pd.read_excel(os.path.join(directory,folder, file))

price.sort_values(by=['Location', 'Grade Type','Date'], inplace=True)
price['End Date'] = price.groupby(['Location','Grade Type'])['Date'].shift(-1)
max_date = max(price['Date'])
price['End Date'] = price['End Date'].fillna(max_date.date().strftime('%Y-%m-%d'))
price['Date_Range'] = price.apply(lambda x: daterange(x['Date'], x['End Date']), axis=1)
price = price.explode('Date_Range')
price = price[price['Date_Range'] < price['End Date']]
price = price[['Location','Grade Type','Date_Range','Fuel Price']]
agg = {      
   'Fuel Price': 'mean'
}
price = price.groupby(['Grade Type','Date_Range']).aggregate(agg).reset_index()
price.drop_duplicates(inplace = True)

#writing Price
filename = 'price.csv'
write_path = os.path.join(directory,subfolder,subfolder_1,filename)
price.to_csv(write_path)

#Adding Price Details
full_data_timeseries = pd.merge(full_data_timeseries,price, left_on = ['Grade Type','date'], right_on = ['Grade Type','Date_Range'], how = 'left')
full_data_timeseries = full_data_timeseries.drop('Date_Range', axis = 1)

In [20]:
full_data_timeseries.shape
# directory = r"C:\Users\Shraddha.Mishra\OneDrive - Shell\Career_development\Projects\Ultimate Potential South Africa"

(2624953, 22)

In [22]:
#Reading Loyalty Programes
agg = {      
   'Estimated Budget in ZAR': 'sum',
    'Campaign Description':'nunique'
}

folder = 'External Variables'
file = 'SA Retail Loyalty Data.xlsx'
loyalty = pd.read_excel(os.path.join(directory,folder, file))
loyalty = loyalty[['Campaign Description','Estimated Budget in ZAR','Start Date','End Date']]


loyalty['Estimated Budget in ZAR'] = pd.to_numeric(loyalty['Estimated Budget in ZAR'], errors='coerce')
loyalty['Campaign Description'] = loyalty['Campaign Description'].astype(str)

print("Loyalty data types after conversion:")
    # print(loyalty.dtypes)
    # print(loyalty['Estimated Budget in ZAR'].unique())
    
loyalty = loyalty.dropna(subset=['Estimated Budget in ZAR'])

loyalty['Date_Range'] = loyalty.apply(lambda x: daterange(x['Start Date'], x['End Date']), axis=1)
loyalty = loyalty.explode('Date_Range')
loyalty['Estimated Budget in ZAR']
loyalty = loyalty[['Campaign Description','Estimated Budget in ZAR','Date_Range']]
loyalty = loyalty.groupby('Date_Range').aggregate(agg).reset_index()

#Reading Marketing Programes
agg = {      
   'Estimated Budget in USD': 'sum',
    'Potential Impact Percentage': 'mean',
    'Campaign Name/ID':'nunique'
}
folder = 'External Variables'
file = 'SA Retail Marketing Data.xlsx'
marketing = pd.read_excel(os.path.join(directory,folder, file))
marketing = marketing[['Campaign Name/ID','Estimated Budget in USD','Potential Impact Percentage','Start Date','End Date']]
marketing['Date_Range'] = marketing.apply(lambda x: daterange(x['Start Date'], x['End Date']), axis=1)
marketing = marketing.explode('Date_Range')
marketing = marketing[['Campaign Name/ID','Estimated Budget in USD','Potential Impact Percentage','Date_Range']]
marketing = marketing.groupby('Date_Range').aggregate(agg).reset_index()

#writing Loyalty
filename = 'loyalty.csv'
write_path = os.path.join(directory,subfolder,subfolder_1,filename)
loyalty.to_csv(write_path)

#writing Marketing
filename = 'marketing.csv'
write_path = os.path.join(directory,subfolder,subfolder_1,filename)
marketing.to_csv(write_path)

#Adding Loyalty
full_data_timeseries = pd.merge(full_data_timeseries,loyalty, left_on = ['date'], right_on = ['Date_Range'], how = 'left')
full_data_timeseries['Loyalty Flag'] = full_data_timeseries['Date_Range'].apply(lambda x: 0 if pd.isnull(x) else 1)
full_data_timeseries = full_data_timeseries.drop('Date_Range', axis = 1)

#Marketing
full_data_timeseries = pd.merge(full_data_timeseries,marketing, left_on = ['date'], right_on = ['Date_Range'], how = 'left')
full_data_timeseries['Marketing Flag'] = full_data_timeseries['Date_Range'].apply(lambda x: 0 if pd.isnull(x) else 1)
full_data_timeseries = full_data_timeseries.drop('Date_Range', axis = 1)

Loyalty data types after conversion:


In [23]:
full_data_timeseries.shape

(2624953, 29)

In [24]:
#Writing the Data
folder = 'EDA'
filename = 'ProcessedData.csv'
write_path = os.path.join(directory,folder,filename)
full_data_timeseries.to_csv(write_path)

In [25]:
full_data_timeseries.shape

(2624953, 29)

In [26]:
full_data_timeseries[full_data_timeseries['date'] == '2019-01-01'].to_csv(os.path.join(directory,folder,'Check.csv'))

## Exploratory Data analysis

In [27]:
agg = {      
    'Sales Volumes in L15': 'sum',
    'Holiday Flag': 'max',
    'Holiday Desc': 'max'
}
holidays_eda = full_data_timeseries.groupby(['Material Number','date']).aggregate(agg).reset_index()

agg = {      
    'Sales Volumes in L15': 'sum',
    'LockDown Flag': 'max'
}
lockdowns_eda = full_data_timeseries.groupby(['Material Number','Province','date']).aggregate(agg).reset_index()

agg = {      
    'Sales Volumes in L15': 'sum',
    'Fuel Price': 'mean'
}
price_eda = full_data_timeseries.groupby(['Material Number','date']).aggregate(agg).reset_index()

agg = {      
    'Sales Volumes in L15': 'sum',
    'Loyalty Flag': 'max',
    'Estimated Budget in ZAR': 'max'
    
}
loyalty_eda = full_data_timeseries.groupby(['Material Number','date']).aggregate(agg).reset_index()

agg = {      
    'Sales Volumes in L15': 'sum',
    'Marketing Flag': 'max',
    'Estimated Budget in USD': 'max'
    
}
marketing_eda = full_data_timeseries.groupby(['Material Number','date']).aggregate(agg).reset_index()

In [28]:
holidays_eda['Month'] = holidays_eda.date.dt.month
holidays_eda['Day'] = holidays_eda['date'].dt.dayofyear

lockdowns_eda['Month'] = lockdowns_eda.date.dt.month
lockdowns_eda['Day'] = lockdowns_eda['date'].dt.dayofyear

price_eda['Month'] = price_eda.date.dt.month
price_eda['Day'] = price_eda['date'].dt.dayofyear

loyalty_eda['Month'] = loyalty_eda.date.dt.month
loyalty_eda['Day'] = loyalty_eda['date'].dt.dayofyear

marketing_eda['Month'] = marketing_eda.date.dt.month
marketing_eda['Day'] = marketing_eda['date'].dt.dayofyear

In [29]:
folder = 'EDA'
filename = 'Holidays_EDA.csv'
write_path = os.path.join(directory,folder,filename)
holidays_eda.to_csv(write_path)

folder = 'EDA'
filename = 'Lockdowns_EDA.csv'
write_path = os.path.join(directory,folder,filename)
lockdowns_eda.to_csv(write_path)

folder = 'EDA'
filename = 'FuelPrice_EDA.csv'
write_path = os.path.join(directory,folder,filename)
price_eda.to_csv(write_path)

folder = 'EDA'
filename = 'Loyalty_EDA.csv'
write_path = os.path.join(directory,folder,filename)
loyalty_eda.to_csv(write_path)

folder = 'EDA'
filename = 'Marketing_EDA.csv'
write_path = os.path.join(directory,folder,filename)
marketing_eda.to_csv(write_path)